In [ ]:
# imports
import torch
from torch import nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
#####################
# my files
# target model
#from net_ import target_net
from net_conv import target_net
# gan architectures
import gans_archs
# advgan training class
from GAN_ import advGAN

if torch.cuda.is_available():  
    dev = 'cuda:0'
else:  
    dev = 'cpu'

print('device: ', dev)

# functions
def transform_data(data_loader_obj):
    ims = []
    lls = []
    for imgs,lbls in data_loader_obj:
        for img,lbl in zip(imgs,lbls):
            ims.append(img)
            lls.append(lbl)
    ims = torch.stack(ims)
    lls = torch.stack(lls)
    return ims,lls

In [ ]:
# obtain mnist data and process
batch_size = 128
num_of_classes = 10

def get_indices(dataset,ind_array):
    indices =  []
    for i in range(len(dataset.targets)):
        for ind in ind_array:
            if dataset.targets[i] == ind:
                indices.append(i)
    return indices

dataset = MNIST('.', train=True, download=False,transform=transforms.ToTensor())

idx = get_indices(dataset, np.arange(num_of_classes))
data_loader_rftarget = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))
data_loader_target = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))
data_loader_gan = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))
data_loader_test = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))

classes = np.arange(num_of_classes)

In [ ]:
# target model
n_estimators = 500
ims_train,lls_train = transform_data(data_loader_rftarget)
rf = RandomForestClassifier(n_estimators=n_estimators)
#rf.fit(ims_train.reshape(len(ims_train),28*28).detach().numpy(),lls_train)
# save the model
PATH = './target_models/rf_model'+str(num_of_classes)+'classes'+'_nest'+str(n_estimators)+'.joblib'
#dump(rf, PATH) 
# load the model
rf = load(PATH)

# test random forest accuracy
ims_test,lls_test = transform_data(data_loader_gan)
print('accuracy: ', accuracy_score(rf.predict(ims_test.reshape(len(ims_test),28*28).detach().numpy()),lls_test))

In [ ]:
# train distilled neural network on output from random forest
net = target_net(num_of_classes).to(dev)
criterion_tar = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

PATH = './target_models/conv_net_distilled_'+str(num_of_classes)+'classes_device-'+dev+'.pth'
# train and and save the model
net.train(data_loader_target, criterion_tar, optimizer, dev, master_model=rf.predict)
torch.save(net.state_dict(), PATH)
# load the model
net = target_net(num_of_classes).to(dev)
net.load_state_dict(torch.load(PATH))

print('model accuracy: ', net.accuracy(data_loader_gan,dev))

In [ ]:
# import gen/disc
gen = gans_archs.Generator2()
disc = gans_archs.Discriminator2()

# arguments for GAN training 
target_net, gen, disc,
tar_criterion=nn.CrossEntropyLoss()
criterion=nn.BCEWithLogitsLoss()
n_epochs=200
batch_size=128
lr=0.00001
device=dev
display_step=500
gen_arch='cov'
###############################
gen_arch_num=2
disc_coeff=1850.
hinge_coeff=50.
adv_coeff=200.
c=0.2
gen_path_extra='distilledrf_genarch_'+str(gen_arch_num)
shape=(1,28,28)
num_of_classes=num_of_classes
################################

# initiate advgan
advgan = advGAN(net.predict_proba,gen,disc,tar_criterion=tar_criterion,
                criterion=criterion,n_epochs=n_epochs,
                batch_size=batch_size,num_of_classes=num_of_classes,
                lr=lr,disc_coeff=disc_coeff,hinge_coeff=hinge_coeff,
                adv_coeff=adv_coeff,c=c,gen_path_extra=gen_path_extra,
                device=device,display_step=display_step,shape=shape,gen_arch=gen_arch)

In [ ]:
%%time 
# train the gan
gen,disc = advgan.train(data_loader_gan)

In [ ]:
from sklearn.metrics import accuracy_score
# test transferability from adv examples for the nn to the rf
counter = 0.
total_rf_clean = 0.
total_nn_clean = 0.
total_rf_adv = 0.
total_nn_adv = 0.
for data,label in DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx)):
    rf_output_clean = rf.predict(data.reshape(len(data),28*28).detach().numpy())
    nn_output_clean = torch.argmax(net(data.reshape(len(data),28*28)),dim=1)
    # make adv example
    pert = gen(data.reshape(len(data),28*28))
    adv_img = data.reshape(len(data),28*28) + pert
    rf_output_adv = rf.predict(adv_img.detach().numpy())
    nn_output_adv = torch.argmax(net(adv_img),dim=1)
    # calculate and return accuracy 
    total_rf_clean += accuracy_score(rf_output_clean,label)
    total_nn_clean += accuracy_score(nn_output_clean,label)
    total_rf_adv += accuracy_score(rf_output_adv,label)
    total_nn_adv += accuracy_score(nn_output_adv,label)
    counter += 1.
print('rf accuracy (clean): ', total_rf_clean/counter)
print('nn accuracy (clean): ', total_nn_clean/counter)
print('% adv rf: ', 1.-total_rf_adv/counter)
print('% adv nn: ', 1.-total_nn_adv/counter)